In [2]:
using Pkg
using Revise
Pkg.activate(dirname(dirname(@__DIR__)))
using Macro
using Gurobi
using Plots
using DataFrames, CSV
using BenchmarkTools
using JuMP

  Activating project at `~/Macro`


In [3]:
case_path = @__DIR__
println("###### ###### ######")
println("Running case at $(case_path)")

system = Macro.load_system(case_path)

model = Macro.generate_model(system)

Macro.set_optimizer(model, Gurobi.Optimizer);

Macro.set_optimizer_attributes(model, "BarConvTol"=>1e-3,"Crossover" => 0, "Method" => 2)

Macro.optimize!(model)

###### ###### ######
Running case at /home/al3792/Macro/ExampleSystems/three_zones_macro_genx_no_cement


┌ Info: Loading JSON data from /home/al3792/Macro/ExampleSystems/three_zones_macro_genx_no_cement/system_data.json
└ @ Macro /home/al3792/Macro/src/load_inputs/load_macroobject.jl:135
┌ Info: Loading JSON data from /home/al3792/Macro/ExampleSystems/three_zones_macro_genx_no_cement/system_data.json
└ @ Macro /home/al3792/Macro/src/load_inputs/load_macroobject.jl:135
┌ Info: Loading JSON data from system/nodes.json
└ @ Macro /home/al3792/Macro/src/load_inputs/load_macroobject.jl:135
┌ Info: Loading CSV data from system/demand.csv
└ @ Macro /home/al3792/Macro/src/load_inputs/file_io/csv.jl:12
┌ Info: Loading CSV data from system/demand.csv
└ @ Macro /home/al3792/Macro/src/load_inputs/file_io/csv.jl:14
┌ Info: Loading JSON data from assets/power_lines.json
└ @ Macro /home/al3792/Macro/src/load_inputs/load_macroobject.jl:135
┌ Info: Loading JSON data from assets/storage.json
└ @ Macro /home/al3792/Macro/src/load_inputs/load_macroobject.jl:135
┌ Info: Loading JSON data from assets/thermal.js

system.settings.Scaling = false

┌ Info: Loading CSV data from assets/availability.csv
└ @ Macro /home/al3792/Macro/src/load_inputs/file_io/csv.jl:14


┌ Info: Starting model generation
└ @ Macro /home/al3792/Macro/src/generate_model.jl:3
┌ Info: Adding linking variables
└ @ Macro /home/al3792/Macro/src/generate_model.jl:17
┌ Info: Defining available capacity
└ @ Macro /home/al3792/Macro/src/generate_model.jl:20
┌ Info: Generating planning model
└ @ Macro /home/al3792/Macro/src/generate_model.jl:23
┌ Info: Generating operational model
└ @ Macro /home/al3792/Macro/src/generate_model.jl:26
┌ Info: Model generation complete, it took 17.675588130950928 seconds
└ @ Macro /home/al3792/Macro/src/generate_model.jl:31


Set parameter LicenseID to value 197246
Set parameter GURO_PAR_SPECIAL
Set parameter TokenServer to value "license.rc.princeton.edu"
Set parameter BarConvTol to value 0.001
Set parameter Crossover to value 0
Set parameter Method to value 2
Set parameter Method to value 2
Set parameter Crossover to value 0
Set parameter BarConvTol to value 0.001
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (linux64 - "Red Hat Enterprise Linux 8.10 (Ootpa)")

CPU model: Intel(R) Xeon(R) Gold 6246R CPU @ 3.40GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 32 physical cores, 32 logical processors, using up to 32 threads

Optimize a model with 648257 rows, 394231 columns and 2285307 nonzeros
Model fingerprint: 0xec24e63c
Coefficient statistics:
  Matrix range     [4e-07, 2e+04]
  Objective range  [1e-01, 2e+07]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+04]
Presolve removed 167078 rows and 61957 columns
Presolve time: 0.81s
Presolved: 481179 rows, 332274 columns, 1760002 n

In [7]:
case = "capacity_no_co2_cap"

capacity_results = Macro.get_optimal_asset_capacity(system)
results_dir = joinpath(case_path, "results")
mkpath(results_dir)
Macro.write_csv(joinpath(results_dir, case * "_capacity.csv"), capacity_results)
println("wrote results")

wrote results


In [15]:
co2_nodes = Macro.get_nodes_sametype(system.locations, CO2)

total_co2_emissions_df = DataFrame(
    node_1_emissions = Macro.value(sum(co2_nodes[1].operation_expr[:exogenous])),
    node_2_emissions = Macro.value(sum(co2_nodes[2].operation_expr[:exogenous])),
    node_3_emissions = Macro.value(sum(co2_nodes[3].operation_expr[:exogenous]))
)
Macro.write_csv(joinpath(results_dir, case * "_co2_emissions.csv"), total_co2_emissions_df)

"/home/al3792/Macro/ExampleSystems/three_zones_macro_genx_no_cement/results/capacity_no_co2_cap_co2_emissions.csv"

In [11]:
co2_emissions_df = DataFrame(
    ng_1_co2 = Macro.value.(Macro.flow(system.assets[6].co2_edge)).data,
    ng_2_co2 = Macro.value.(Macro.flow(system.assets[7].co2_edge)).data,
    ng_3_co2 = Macro.value.(Macro.flow(system.assets[8].co2_edge)).data
)

Macro.write_csv(joinpath(results_dir, case * "_co2_emissions_8760.csv"), co2_emissions_df)

"/home/al3792/Macro/ExampleSystems/three_zones_macro_genx_no_cement/results/capacity_no_co2_cap_co2_emissions_8760.csv"

In [13]:
sum.(eachcol(co2_emissions_df))

3-element Vector{Float64}:
     1.873906929961118e7
     2.4672028562540933e6
 34485.34859300121

In [23]:
sum(sum.(eachcol(co2_emissions_df)))

2.1240757504458275e7